In [ ]:
# 기본 옵션
options(warn = -1)
options(repr.plot.width=15, repr.plot.height=9)
options(scipen = 100)
Sys.setlocale(category = "LC_ALL", locale = "ko_KR.UTF-8")

# library 불러오기
library(stringr)
library(plotrix)
library(dplyr)

In [ ]:
basic <- function(parameters) {
   str(parameters)
   summary(parameters)
}

In [ ]:
election <- read.csv("Data/election_2022.csv")

In [ ]:
# electionSum = 이재명 + 윤석열 + 심상정
election$electionSum <- election$이재명 + election$윤석열 + election$심상정
election$electionSum <- apply(election[5:7],1,sum)

In [ ]:
# electionSum에 투표수 몇 percent?
election$electionRatio <- election$electionSum / election$투표수 * 100

In [ ]:
# electionRatio가 가장 높은 시군구와 가장 낮은 시군구?
# 1) arrange
arrange(election,desc(electionRatio))[c(1,nrow(election)),c(1:2,ncol(election))]
# 2) subset
subset(election,electionRatio == max(election$electionRatio) | electionRatio == min(election$electionRatio))

In [73]:
# CSV로 저장
write.csv(election,"Data/election_2022_job.csv")